In [1]:
import numpy as np
import tensorflow as tf

from MusicRnnData import MusicRnnData

x_len = 50
y_len = 1
batch_size = 64
lstm_size = 128
hidden_size = 128
learning_rate = 1e-3
num_steps = 50000
display_interval = 100
verbose = True
moving_avg_length = 10

# fix random seed for reproducibility
np.random.seed(7)

NameError: name 'moving_avg_length' is not defined

## Load the data

In [ ]:
filelist = ['a2002011001-e02.wav']
music_data = MusicRnnData(filelist)

## Helper functions

In [ ]:
from collections import deque

class MovingAverager(object):
    def __init__(self, filter_length):
        self.filter = deque([0 for _ in range(filter_length)])
        
    def insert(self, num):
        self.filter.popleft()
        self.filter.append(num)
        
    def average(self):
        return sum(self.filter)/float(len(self.filter))

## Construct model

In [ ]:
# input
x = tf.placeholder(tf.float32, [batch_size, x_len])
y = tf.placeholder(tf.float32, [batch_size, y_len])

# RNN Cell
lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
# add dropout here

# output layer
output, states = tf.contrib.rnn.static_rnn(lstm, [x], dtype=tf.float32)
y_ = output[-y_len:]

## Set cost function and optimizer

In [ ]:
cost = tf.nn.l2_loss(y_ - y)
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

## Run training session

In [ ]:
moving_avg = MovingAverager(moving_avg_length)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(num_steps):
        x_batch, y_batch = music_data.batch(x_len, y_len, batch_size)
        _, loss = sess.run([optimizer, cost], feed_dict={x: x_batch, y: y_batch})

        # print progress
        if verbose and (i % display_interval == 0):
            print('step: %d, training loss: %g' % (i, loss))
            
            # moving averager
            moving_avg.add(loss)
            if i > moving_avg_length:
                print('moving average: %g' % (moving_avg.average))